# Step 1: Imports and Setup

In [1]:
# %%
import duckdb
import pandas as pd
from pathlib import Path

# --- Configuration ---
# Point to the database file you created in the ETL step
OUTPUT_DIR = Path("../output")
DB_FILE = OUTPUT_DIR / "synthea_fhir.duckdb"

# --- Connect to the database ---
# We're connecting in read-only mode since we are just querying, not writing new tables.
print(f"Connecting to DuckDB database: {DB_FILE}")
con = duckdb.connect(database=str(DB_FILE), read_only=True)

# Set pandas to display more columns for our sanity checks
pd.set_option('display.max_columns', 50)

Connecting to DuckDB database: ../output/synthea_fhir.duckdb


# Step 2: Identify Index Admissions

Our goal is to predict readmission following an inpatient hospital stay. We can't predict readmissions for every single encounter (like a regular doctor's visit). We must first create a base table of only the relevant hospitalizations. These are our index admissions.

In the Synthea dataset, inpatient encounters are marked with the code 'IMP' in the EncounterClass column.

In [2]:
# %%
# SQL query to select all inpatient encounters
sql_query = """
SELECT
    Id AS encounter_id,
    Patient AS patient_id,
    Start AS admission_date,
    Stop AS discharge_date
FROM
    encounters
WHERE
    EncounterClass = 'IMP'
"""

# Execute and fetch into a pandas DataFrame
index_admissions_df = con.execute(sql_query).fetchdf()

🩺 Sanity Check
Let's verify that we've correctly filtered the data. We should see a reasonable number of inpatient stays and the dates should look correct.

In [3]:
# %%
print(f"Found {len(index_admissions_df):,} total inpatient admissions.")
print("First 5 index admissions:")
display(index_admissions_df.head())

# Check for any admissions with null discharge dates, as these are problematic
print(f"\nAdmissions with missing discharge dates: {index_admissions_df['discharge_date'].isnull().sum()}")

Found 104,067 total inpatient admissions.
First 5 index admissions:


,encounter_id,patient_id,admission_date,discharge_date
0,d9f75434-be7a-663f-8702-8ac72afe10fd,7b0e0003-89c8-6020-da41-033cf174c76f,1997-05-18 10:35:30,1997-05-23 08:41:12
1,58fa79a0-fb0a-9eb3-562a-ece296e084f6,e8e952cb-c390-f661-ad04-b95818ec40a4,1994-05-21 00:52:34,1994-05-25 19:53:04
2,8be2ca69-121a-a779-3ac9-2ecb97ad8908,49b95517-d094-9b08-3982-c86d39450368,2025-04-03 20:25:55,2025-04-04 20:40:55
3,b53c2a1a-865d-c845-213a-6e1ffada051c,96343d6e-0394-1afb-2162-f7a088d446af,2015-02-23 02:28:34,2015-02-24 20:17:51
4,a7b6dfe0-dda9-fb85-0cda-2494f814f28d,96343d6e-0394-1afb-2162-f7a088d446af,2024-09-05 00:28:34,2024-09-09 00:43:34



Admissions with missing discharge dates: 0


# Step 3: Engineer the Target Variable (readmitted_within_30_days)

This is the most critical step. For each index admission, we need to determine if another inpatient admission for the same patient occurred within 30 days of their discharge.

We can do this efficiently using a SQL window function (LEAD) to find the next admission date for each patient.

In [4]:
# %%
# This query calculates the time to the next admission and creates the readmission flag.
sql_query = """
WITH PatientAdmissions AS (
    -- 1. Get all inpatient admissions and order them by patient and date
    SELECT
        Id AS encounter_id,
        Patient AS patient_id,
        Start AS admission_date,
        Stop AS discharge_date,
        -- 2. Use the LEAD window function to get the start date of the *next* admission
        -- for the same patient. If there is no next admission, it will be NULL.
        LEAD(Start, 1) OVER(PARTITION BY Patient ORDER BY Start) AS next_admission_date
    FROM
        encounters
    WHERE
        EncounterClass = 'IMP'
)
-- 3. Now, calculate the difference and create the flag
SELECT
    encounter_id,
    patient_id,
    admission_date,
    discharge_date,
    next_admission_date,
    -- Calculate days between discharge and the next admission
    DATE_DIFF('day', discharge_date, next_admission_date) AS days_to_next_admission,
    -- Create the binary target variable
    CASE
        WHEN DATE_DIFF('day', discharge_date, next_admission_date) <= 30 THEN 1
        ELSE 0
    END AS readmitted_within_30_days
FROM
    PatientAdmissions
-- We can't predict readmission for ongoing hospitalizations
WHERE
    discharge_date IS NOT NULL
"""

# This will be our main dataset that we'll add features to
readmissions_df = con.execute(sql_query).fetchdf()

🩺 Sanity Check
Let's check the distribution of our new target variable. We should also manually inspect a few positive cases to be sure our logic is correct.

In [5]:
# %%
# Check the distribution of the target variable
print("Readmission distribution:")
print(readmissions_df['readmitted_within_30_days'].value_counts(normalize=True))

# Find a patient who was readmitted and verify the dates manually
readmitted_patient_example = readmissions_df[readmissions_df['readmitted_within_30_days'] == 1].head(5)
print("\nExample of patients who were readmitted:")
display(readmitted_patient_example)

Readmission distribution:
readmitted_within_30_days
0    0.821903
1    0.178097
Name: proportion, dtype: float64

Example of patients who were readmitted:


,encounter_id,patient_id,admission_date,discharge_date,next_admission_date,days_to_next_admission,readmitted_within_30_days
4,da9b2e6d-dc94-7418-d9f8-08c9c9bf5158,52f67a19-eb1d-7951-e3ac-685bd96093a2,1985-08-04 05:26:15,1985-08-05 05:26:15,1985-08-06 11:26:50,1,1
42,fd7a4191-9837-4d3f-fde7-627448a066a4,550874b4-44b6-0359-aacf-cb038907cea7,2022-12-06 21:34:16,2022-12-24 08:55:16,2023-01-22 21:55:16,29,1
44,2ab4084c-5fd8-abeb-bb70-cfb9051e34c5,550874b4-44b6-0359-aacf-cb038907cea7,2023-02-28 11:07:16,2023-03-04 18:10:16,2023-04-03 20:10:16,30,1
47,b8e07660-bf74-bf09-eb8c-c4091d5235eb,550874b4-44b6-0359-aacf-cb038907cea7,2023-06-22 05:00:16,2023-06-28 18:12:16,2023-07-27 01:12:16,29,1
52,4e4c504c-a663-731d-2b0c-a917644e8298,550874b4-44b6-0359-aacf-cb038907cea7,2024-01-08 01:08:16,2024-01-19 10:25:16,2024-02-17 11:25:16,29,1


# Step 4: Add Demographics and Admission-Level Features

Now we can start building our features. We'll join our readmissions_df with the patients table to get demographic data and also calculate some features from the admission itself.

In [6]:
# %%
sql_query = """
SELECT
    -- Key identifiers from our readmissions table
    readmissions.encounter_id,
    readmissions.patient_id,

    -- The target variable
    readmissions.readmitted_within_30_days,

    -- Feature 1: Length of Stay (in days)
    DATE_DIFF('day', readmissions.admission_date, readmissions.discharge_date) AS length_of_stay,

    -- Feature 2: Age at time of admission
    DATE_DIFF('year', patients.BirthDate, readmissions.admission_date) AS age_at_admission,

    -- Demographic features from the patients table
    patients.Gender AS gender,
    patients.Race AS race,
    patients.Marital AS marital_status,
    
    -- Admission details from the original encounters table
    enc.Description AS admission_reason,
    enc.ReasonDescription AS admission_reason_detail,
    enc.Payer AS payer,
    enc.Total_Claim_Cost AS total_claim_cost,
    patients.Income AS income,
    DAYNAME(readmissions.admission_date) AS admission_day_of_week,

    -- === NEW HIGH-CARDINALITY FEATURES ===

    -- 1. Primary Diagnosis Code (from Encounter ReasonCode)
    enc.ReasonCode AS primary_diagnosis_code,

    -- 2. Attending Provider ID
    enc.Provider AS provider_id

FROM
    -- Use our previously created DataFrame as a source table
    readmissions_df AS readmissions
LEFT JOIN
    patients ON readmissions.patient_id = patients.Id
LEFT JOIN
    encounters AS enc ON readmissions.encounter_id = enc.Id
"""

# Create our analytical base table
model_df = con.execute(sql_query).fetchdf()

In [7]:
# Create the new interaction feature by combining two columns
# We convert to string and fill NAs to prevent errors
model_df['payer_dx_interaction'] = (
    model_df['payer'].astype(str).fillna('unknown') + '_' + 
    model_df['primary_diagnosis_code'].astype(str).fillna('unknown')
)

print("New high-cardinality features added:")
display(model_df[['primary_diagnosis_code', 'provider_id', 'payer_dx_interaction']].head())

New high-cardinality features added:


,primary_diagnosis_code,provider_id,payer_dx_interaction
0,401314000,us-npi|9999868992,Medicare_401314000
1,274531002,us-npi|9999868992,Aetna_274531002
2,399261000,us-npi|9999868992,Aetna_399261000
3,399261000,us-npi|9999868992,Aetna_399261000
4,6525002,us-npi|9999927293,Anthem_6525002


🩺 Sanity Check
Let's inspect the resulting dataframe. The row count should be the same as our readmissions_df. We should also check the distributions of our new numeric features like age_at_admission and length_of_stay to see if they are reasonable.

In [8]:
# %%
print(f"Total rows in our model dataset: {len(model_df):,}")
print("First 5 rows of the feature table:")
display(model_df.head())

print("\nStatistical summary of numeric features:")
display(model_df[['length_of_stay', 'age_at_admission']].describe())

Total rows in our model dataset: 104,067
First 5 rows of the feature table:


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail,payer,total_claim_cost,income,admission_day_of_week,primary_diagnosis_code,provider_id,payer_dx_interaction
0,fd70210f-52cb-77ef-61c7-aadbd1018a99,e3d3ed64-ca29-b95b-dbba-abfa3ee61b44,0,1,69,male,White,D,Hospital admission (procedure),Acute non-ST segment elevation myocardial infa...,Medicare,40610.128906,33853,Sunday,401314000,us-npi|9999868992,Medicare_401314000
1,f2fa6d45-a393-6aea-83d4-166fbaa0a4a9,34270d54-36d4-f26b-4e4d-0cdb327cf159,1,1,57,male,White,D,Hospital admission (procedure),Abnormal findings diagnostic imaging heart+cor...,Aetna,37202.929688,30294,Thursday,274531002,us-npi|9999868992,Aetna_274531002
2,3c16cdb1-a750-7963-b963-cc0f90e727f1,34270d54-36d4-f26b-4e4d-0cdb327cf159,1,2,57,male,White,D,Patient transfer to intensive care unit (proce...,History of coronary artery bypass grafting (si...,Aetna,15647.690430,30294,Thursday,399261000,us-npi|9999868992,Aetna_399261000
3,3fc36a47-cb8f-56a4-3b11-a434126d40ca,34270d54-36d4-f26b-4e4d-0cdb327cf159,0,1,57,male,White,D,Admission to ward (procedure),History of coronary artery bypass grafting (si...,Aetna,2176.229980,30294,Saturday,399261000,us-npi|9999868992,Aetna_399261000
4,df4e5b66-ca7f-5b68-a5c2-c77def0a94ce,13aebc7f-8e4a-396d-1666-1af41cb27781,0,13,43,male,White,S,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),Anthem,119.830002,11631,Monday,6525002,us-npi|9999927293,Anthem_6525002



Statistical summary of numeric features:


,length_of_stay,age_at_admission
count,104067.000000,104067.000000
mean,6.917226,50.162895
std,14.102997,19.497269
min,0.000000,0.000000
25%,1.000000,36.000000
50%,4.000000,52.000000
75%,8.000000,65.000000
max,3306.000000,111.000000


# Step 5: Engineer Historical Features (Advanced)

In [9]:
# %%
# This query is more complex. It performs a "self-join" on the encounters
# table to count previous visits for each index admission.
sql_query = """
SELECT
    index_admission.Id AS encounter_id,
    -- Count all previous inpatient admissions within the last 365 days
    COUNT(prior_admissions.Id) AS prior_admissions_last_year
FROM
    encounters AS index_admission
LEFT JOIN
    encounters AS prior_admissions
ON
    -- Must be the same patient
    index_admission.Patient = prior_admissions.Patient
    -- The prior admission must have happened BEFORE the index one
    AND prior_admissions.Start < index_admission.Start
    -- And it must be within the last year
    AND DATE_DIFF('day', prior_admissions.Start, index_admission.Start) <= 365
    -- And it must also be an inpatient admission
    AND prior_admissions.EncounterClass = 'IMP'
WHERE
    -- We only need to calculate this for our index admissions
    index_admission.EncounterClass = 'IMP'
GROUP BY
    index_admission.Id;
"""

prior_admissions_df = con.execute(sql_query).fetchdf()

# Now, merge this back into our main model dataframe
model_df = pd.merge(model_df, prior_admissions_df, on='encounter_id', how='left')

# Fill any NaNs that might result from the merge with 0
model_df['prior_admissions_last_year'] = model_df['prior_admissions_last_year'].fillna(0)

🩺 Sanity Check
Let's check the new column. The values should be integers, and the distribution should make sense (most patients will likely have 0 or 1 prior admissions).

In [10]:
# %%
print("Distribution of prior admissions in the last year:")
print(model_df['prior_admissions_last_year'].value_counts().sort_index())

display(model_df.head())

Distribution of prior admissions in the last year:
prior_admissions_last_year
0     66545
1     10832
2      3184
3      1464
4       860
5       804
6       928
7      1214
8      3005
9     10674
10     1931
11     1944
12      602
13       64
14       16
Name: count, dtype: int64


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail,payer,total_claim_cost,income,admission_day_of_week,primary_diagnosis_code,provider_id,payer_dx_interaction,prior_admissions_last_year
0,fd70210f-52cb-77ef-61c7-aadbd1018a99,e3d3ed64-ca29-b95b-dbba-abfa3ee61b44,0,1,69,male,White,D,Hospital admission (procedure),Acute non-ST segment elevation myocardial infa...,Medicare,40610.128906,33853,Sunday,401314000,us-npi|9999868992,Medicare_401314000,0
1,f2fa6d45-a393-6aea-83d4-166fbaa0a4a9,34270d54-36d4-f26b-4e4d-0cdb327cf159,1,1,57,male,White,D,Hospital admission (procedure),Abnormal findings diagnostic imaging heart+cor...,Aetna,37202.929688,30294,Thursday,274531002,us-npi|9999868992,Aetna_274531002,0
2,3c16cdb1-a750-7963-b963-cc0f90e727f1,34270d54-36d4-f26b-4e4d-0cdb327cf159,1,2,57,male,White,D,Patient transfer to intensive care unit (proce...,History of coronary artery bypass grafting (si...,Aetna,15647.690430,30294,Thursday,399261000,us-npi|9999868992,Aetna_399261000,1
3,3fc36a47-cb8f-56a4-3b11-a434126d40ca,34270d54-36d4-f26b-4e4d-0cdb327cf159,0,1,57,male,White,D,Admission to ward (procedure),History of coronary artery bypass grafting (si...,Aetna,2176.229980,30294,Saturday,399261000,us-npi|9999868992,Aetna_399261000,2
4,df4e5b66-ca7f-5b68-a5c2-c77def0a94ce,13aebc7f-8e4a-396d-1666-1af41cb27781,0,13,43,male,White,S,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),Anthem,119.830002,11631,Monday,6525002,us-npi|9999927293,Anthem_6525002,0


# Step 5b: Engineer Clinical Features (Diagnoses, Procedures, Medications)

In [11]:
# %%
# ----- 1. Number of Diagnoses (Conditions) -----
print("Engineering feature: Number of Diagnoses...")
sql_diagnoses = """
SELECT
    Encounter AS encounter_id,
    COUNT(Code) AS num_diagnoses
FROM
    conditions
GROUP BY
    Encounter;
"""
diagnoses_df = con.execute(sql_diagnoses).fetchdf()

# Merge into the main dataframe
model_df = pd.merge(model_df, diagnoses_df, on='encounter_id', how='left')
model_df['num_diagnoses'] = model_df['num_diagnoses'].fillna(0)


# ----- 2. Number of Procedures -----
print("Engineering feature: Number of Procedures...")
sql_procedures = """
SELECT
    Encounter AS encounter_id,
    COUNT(Code) AS num_procedures
FROM
    procedures
GROUP BY
    Encounter;
"""
procedures_df = con.execute(sql_procedures).fetchdf()

# Merge into the main dataframe
model_df = pd.merge(model_df, procedures_df, on='encounter_id', how='left')
model_df['num_procedures'] = model_df['num_procedures'].fillna(0)


# ----- 3. Number of Medications -----
print("Engineering feature: Number of Medications...")
sql_medications = """
SELECT
    Encounter AS encounter_id,
    COUNT(Code) AS num_medications
FROM
    medications
GROUP BY
    Encounter;
"""
medications_df = con.execute(sql_medications).fetchdf()

# Merge into the main dataframe
model_df = pd.merge(model_df, medications_df, on='encounter_id', how='left')
model_df['num_medications'] = model_df['num_medications'].fillna(0)

print("✅ Clinical features added successfully.")

Engineering feature: Number of Diagnoses...
Engineering feature: Number of Procedures...
Engineering feature: Number of Medications...
✅ Clinical features added successfully.


🩺 Sanity Check
Let's inspect our dataframe again to ensure the new columns have been added correctly. They should contain integer counts.

In [12]:
# %%
print("First 5 rows with new clinical features:")
display(model_df.head())

print("\nUpdated statistical summary:")
display(model_df[['length_of_stay', 'age_at_admission', 'prior_admissions_last_year', 'num_diagnoses', 'num_procedures', 'num_medications']].describe())

First 5 rows with new clinical features:


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail,payer,total_claim_cost,income,admission_day_of_week,primary_diagnosis_code,provider_id,payer_dx_interaction,prior_admissions_last_year,num_diagnoses,num_procedures,num_medications
0,fd70210f-52cb-77ef-61c7-aadbd1018a99,e3d3ed64-ca29-b95b-dbba-abfa3ee61b44,0,1,69,male,White,D,Hospital admission (procedure),Acute non-ST segment elevation myocardial infa...,Medicare,40610.128906,33853,Sunday,401314000,us-npi|9999868992,Medicare_401314000,0,1.0,0.0,4.0
1,f2fa6d45-a393-6aea-83d4-166fbaa0a4a9,34270d54-36d4-f26b-4e4d-0cdb327cf159,1,1,57,male,White,D,Hospital admission (procedure),Abnormal findings diagnostic imaging heart+cor...,Aetna,37202.929688,30294,Thursday,274531002,us-npi|9999868992,Aetna_274531002,0,1.0,21.0,0.0
2,3c16cdb1-a750-7963-b963-cc0f90e727f1,34270d54-36d4-f26b-4e4d-0cdb327cf159,1,2,57,male,White,D,Patient transfer to intensive care unit (proce...,History of coronary artery bypass grafting (si...,Aetna,15647.690430,30294,Thursday,399261000,us-npi|9999868992,Aetna_399261000,1,0.0,6.0,0.0
3,3fc36a47-cb8f-56a4-3b11-a434126d40ca,34270d54-36d4-f26b-4e4d-0cdb327cf159,0,1,57,male,White,D,Admission to ward (procedure),History of coronary artery bypass grafting (si...,Aetna,2176.229980,30294,Saturday,399261000,us-npi|9999868992,Aetna_399261000,2,0.0,4.0,0.0
4,df4e5b66-ca7f-5b68-a5c2-c77def0a94ce,13aebc7f-8e4a-396d-1666-1af41cb27781,0,13,43,male,White,S,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),Anthem,119.830002,11631,Monday,6525002,us-npi|9999927293,Anthem_6525002,0,0.0,0.0,0.0



Updated statistical summary:


,length_of_stay,age_at_admission,prior_admissions_last_year,num_diagnoses,num_procedures,num_medications
count,104067.000000,104067.000000,104067.000000,104067.000000,104067.000000,104067.000000
mean,6.917226,50.162895,2.039052,0.377978,9.334006,0.183997
std,14.102997,19.497269,3.557033,0.719729,15.938185,0.818715
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,36.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,52.000000,0.000000,0.000000,4.000000,0.000000
75%,8.000000,65.000000,2.000000,1.000000,12.000000,0.000000
max,3306.000000,111.000000,14.000000,8.000000,176.000000,9.000000


# Step 6: Save the Final Dataset & Cleanup

Our feature engineering is done for now! We have a clean, single table ready for the next steps. It's best practice to save this intermediate result to a fast, efficient file format like Parquet.

In [13]:
# %%
# Define the path for our final dataset
MODEL_DATA_FILE = OUTPUT_DIR / "readmissions_dataset.parquet"

# Save to Parquet
model_df.to_parquet(MODEL_DATA_FILE, index=False)

# Close the database connection
con.close()

print(f"✅ Successfully created and saved the feature table with {len(model_df):,} rows and {len(model_df.columns)} columns.")
print(f"Dataset saved to: {MODEL_DATA_FILE}")

✅ Successfully created and saved the feature table with 104,067 rows and 21 columns.
Dataset saved to: ../output/readmissions_dataset.parquet
